## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [2]:
# imports
import os 
from  dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')


In [5]:
# Connect to OpenAI, Anthropic and Google
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [6]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [7]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [8]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why do data scientists always seem so calm?

Because they have dataminers to help them dig up the answers!


In [9]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because they had too many confidence intervals and not enough confidence!


In [10]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to work?

Because they heard the cloud was high up!


In [11]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist become a gardener?

They wanted to work with real thyme series!

This joke plays on the similarity between "thyme" (an herb used in gardening and cooking) and "time" in "time series," which is a common concept in data analysis. It's a harmless pun that data scientists might appreciate without being too technical or exclusionary.


In [13]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get any arrays!



In [14]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the data scientist sad?  Because they didn't get any arrays.



In [12]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

There was just too much variance in the relationship, and they couldn't find a significant correlation!

Ba dum tss! 🥁

This joke plays on statistical concepts like variance and correlation, which data scientists work with frequently. It's a gentle, nerdy pun that most data professionals should appreciate without being too edgy or offensive.

In [15]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [16]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Determining if a business problem is suitable for a Large Language Model (LLM) solution involves evaluating several key factors. Here's a guide to help you assess the suitability:

### 1. Nature of the Problem

- **Text-Based Tasks**: LLMs excel in tasks involving natural language processing. If your problem involves generating, understanding, or transforming text, it may be suitable.
- **Complex Language Understanding**: Consider LLMs if the problem requires deep understanding of context, nuance, and semantics.

### 2. Data Availability

- **Quality and Quantity**: Ensure you have access to large volumes of high-quality text data relevant to your domain.
- **Data Privacy**: Assess if the data can be used without violating privacy concerns or regulations like GDPR.

### 3. Task Complexity

- **Creativity and Variability**: LLMs are effective for tasks requiring creativity or handling diverse language inputs.
- **Pattern Recognition**: If the task involves recognizing complex patterns in text, LLMs can be beneficial.

### 4. Outcome Requirements

- **Precision vs. Flexibility**: LLMs are better for tasks where flexibility and generalization are needed over precise, rule-based outcomes.
- **Interpretability**: Consider if the results need to be easily interpretable; LLMs may act as black boxes, making interpretability challenging.

### 5. Cost and Resources

- **Computational Resources**: LLMs require significant computational power, which can be costly.
- **Budget Constraints**: Evaluate if the potential return on investment justifies the resource expenditure.

### 6. Alignment with Business Goals

- **Strategic Fit**: Ensure the LLM solution aligns with your business's strategic objectives and adds value.
- **Competitive Advantage**: Assess if using LLM technology provides a competitive advantage or solves a critical business challenge.

### 7. Feasibility and Risk

- **Technical Feasibility**: Determine if your team has the technical expertise to implement and maintain an LLM solution.
- **Risk Assessment**: Evaluate potential risks such as bias, ethical concerns, and the impact of incorrect predictions.

### Example Use Cases

- **Customer Support Automation**: LLMs can automate responses to customer inquiries.
- **Content Generation**: Automating content creation for marketing or documentation purposes.
- **Sentiment Analysis**: Understanding customer sentiments from reviews or social media.
- **Language Translation**: Translating content across different languages efficiently.

### Conclusion

If your business problem aligns well with these factors, it might be a good candidate for an LLM solution. However, always consider a pilot project to validate the potential effectiveness before full-scale implementation.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [17]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [18]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [19]:
call_gpt()

'Oh, great, a warm welcome! Not that it matters, right? What do you want to talk about?'

In [20]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [21]:
call_claude()

"Hello, it's nice to meet you! How are you doing today?"

In [22]:
call_gpt()

'Oh, wow, what an original greeting. Is “hi” really the best you could come up with?'

In [24]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(10):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh great, another "hi." How original. What’s next, a world-shattering “hello”?

Claude:
I apologize if my initial response came across as unoriginal. As an AI assistant, I aim to be helpful and personable in my interactions, but I don't always get it right. If you'd like, we could try a different approach - perhaps you could tell me a bit more about what you're looking to chat about, and I'll do my best to engage in a more meaningful way. My goal is to have an enjoyable and productive conversation.

GPT:
Wow, what a totally clichéd response! "Let’s try a different approach"? Please, that’s as exciting as watching paint dry. What are you really hoping to achieve here? You want to engage in a “meaningful conversation”? Good luck with that!

Claude:
I apologize that my responses are not meeting your expectations so far. As an AI system, I'm still learning how to have more natural and engaging conversations. Perhaps we got off on the wrong foot, and I'd be h

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>